- original code: https://github.com/ma2rten/seq2seq

# layers
# 1.1 embedding layer

In [3]:
import numpy as np
from numpy.random import rand

def initialize(dim, init_range):
    return rand(*dim) * init_range

class Embedding:
    
    def __init__(self, vocab_size, embed_size, init_range=1):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.W = initialize((vocab_size, embed_size), init_range=1)
        self.params = [
            ('W', self.W, self.dW)
        ]
    
    def initSequence(self):
        self.t =0
        self.x = {}
        self.dW[:] = 0 # reset
    
    def forward(self, x):
        self.x[self.t] = x
        self.t += 1
        return self.W[x]
    
    def backward(self, delta):
        self.t -= 1
        x = self.x[self.t]
        self.dW[x] += delta

# 1.2 LSTM
- img link: https://sergioskar.github.io/Bitcon_prediction_LSTM/

![](https://sergioskar.github.io/assets/img/posts/lstm_equations.jpg)

In [4]:
def zeros(*dim):
    return np.zeros(dim)

class LSTM:
    
    def __init__(self, input_size, hidden_size, init_range=1, previous=None):
        self.input_size, self.hidden_size = input_size, hidden_size
        
        if previous:
            self.previous = previous
            previous.next = self
        
        def init(x, y):
            return initialize((x, y), init_range)
    
        h, n = hidden_size, input_size
        
        self.W_hi, self.W_hf, self.W_ho, self.W_hj =\
            init(h, h), init(h, h), init(h, h), init(h, h)
        self.W_xi, self.W_xf, self.W_xo, self.W_xj =\
            init(h, n), init(h, n), init(h, n), init(h, n)
        self.b_i, self.b_f, self.b_o, self.b_j =\
            zeros(h), ones(h)*3, zeros(h), zeros(h)
        
        # initialize gradients
        self.dW_hi, self.dW_hf, self.W_ho, self.W_hj =\
            zeros(h, h), zeros(h, h), zeros(h, h), zeros(h, h)
        self.dW_xi, self.dW_xf, self.dW_xo, self.dW_xj =\
            zeros(h, n), zeros(h, n), zeros(h, n), zeros(h, n)
        self.db_i, self.db_f, self.db_o, self.db_j =\
            zeros(h), zeros(h), zeros(h), zeros(h)
        
        # name, param, grad
        self.params = [
            ('W_hi', self.W_hi, self.dW_hi),
            ('W_hf', self.W_hf, self.dW_hf),
            ('W_ho', self.W_ho, self.dW_ho),
            ('W_hj', self.W_hj, self.dW_hj),
            
            ('W_xi', self.W_xi, self.dW_xi),
            ('W_xf', self.W_xf, self.dW_xf),
            ('W_xo', self.W_xo, self.dW_xo),
            ('W_xj', self.W_xj, self.dW_xj),
            
            ('b_i', self.b_i, self.db_i),
            ('b_f', self.b_f, self.db_f),
            ('b_o', self.b_o, self.db_o),
            ('b_j', self.b_j, self.db_j),
        ]
        self.initSequence()
        
    def initSequence(self):
        self.t = 0
        self.x = {}
        self.h = {}
        self.c = {}
        self.ct = {}
        
        self.input_gate = {}
        self.forget_gate = {}
        self.output_gate = {}
        self.cell_update = {}
        
        if has_attr(self, 'previous') :
            self.h[0] = self.previous.h[self.previous.t]
            self.c[0] = self.previous.c[self.previous.t]
        else:
            self.h[0] = zeros(self.hidden_size)
            self.c[0] = zeros(self.hidden_size)
        
        if has_attr(self, 'next'):
            self.dh_prev = self.next.dh_prev
            self.dc_prev = self.next.dc_prev
        else:
            self.dh_prev = zeros(self.hidden_size)
            self.dc_prev = zeros(self.hidden_size)
        
        for name, param, grad in self.params:
            grad[:] = 0
    
    def forward(self, x_t):
        self.t += 1
        
        t = self.t
        h = self.h[t-1]
        
        self.forget_gate[t] = sigmoid(np.dot(self.W_hf, h) + np.dot(self.W_xf, x_t) + self.b_f)
        self.input_gate[t] = sigmoid(np.dot(self.W_hi, h) + np.dot(self.W_xi, x_t) + self.b_i)
        self.output_gate[t] = sigmoid(np.dot(self.W_ho, h) + np.dot(self.W_xo, x_t) + self.b_o)
        
        self.cell_update[t] = tanh(np.dot(self.W_hj, h) + np.dot(self.W_xj, x_t) + self.b_j)
            
        self.c[t] = self.forget_gate[t] * self.c[t-1] + self.input_gate[t] * self.cell_update[t]
        self.h[t] = self.output_gate[t] * tanh(self.c[t])
        
        self.x[t] = x_t
        return self.h[t]
    
    def backward(self, dh):
        
        t = self.t
        
        dh = dh + self.dh_prev
        dC = tanh_grad(self.ct[t]) * self.output_gate[t] * dh + self.dc_prev
        
        # gate backprop
        d_input = sigmoid_grad(self.input_gate[t]) * self.cell_update[t] * dC
        d_forget = sigmoid_grad(self.forget_gate[t]) * self.c[t-1] * dC
        d_output = sigmoid_grad(self.output_gate[t]) * self.tanh(self.c[t]) * dh
        d_update = tanh_grad(self.cell_update[t]) * self.input_gate[t] * dC
        
        self.dc_prev = self.forget_gate[t] * dC
        
        # bias backprop
        self.db_i += d_input
        self.db_f += d_forget
        self.db_o += d_output
        self.db_j += d_update
        
        h_in = self.h[t-1]
        
        self.dW_xi += np.outer(d_input, self.x[t])
        self.dW_xf += np.outer(d_forget, self.x[t])
        self.dW_xo += np.outer(d_output, self.x[t])
        self.dW_xj += np.outer(d_update, self.x[t])
        
        self.dW_hi += np.outer(d_input, h_in)
        self.dW_hf += np.outer(d_forget, h_in)
        self.dW_ho += np.outer(d_outer, h_in)
        self.dW_hj += np.outer(d_update, h_in)
        
        self.dh_prev = np.dot(self.W_hi.T, d_input)
        self.dh_prev += np.dot(self.W_hf.T, d_forget)
        self.dh_prev += np.dot(self.W_ho.T, d_output)
        self.dh_prev += np.dot(self.W_hj.T, d_update)
        
        dX = np.dot(self.W_xi.T, d_input)
        dX += np.dot(self.W_xf.T, d_forget)
        dX += np.dot(self.W_xo.T, d_output)
        dX += np.dot(self.W_xj.T, d_update)
        
        self.t -= 1
        
        return dX

# 1.3 softmax

In [5]:
class Softmax:
    def __init__(self, input_size, output_size, init_range=1.0):
        self.input_size = input_size
        self.output_size = output_size
        
        self.W = initilize((output_size, input_size), init_range)
        self.dW = np.zeros((output_size, input_size))
        
        self.params = [
            ('W', self.W, self.dW)
        ]
        
    def initSequence(self):
        self.pred = []
        self.x = []
        self.targets = []
        self.t = 0
        self.dW[:] = 0
        
    def forward(self, x):
        self.t += 1
        
        y = self.W.dot(x)
        y = np.exp(y - y.max())
        y /= y.sum()
        
        self.pred.append(y)
        self.x.append(x)
        
        return y
    
    def backward(self, target):
        self.t -= 1
        self.targets.append(target)
        
        x = self.x[self.t]
        d = self.pred[self.t].copy()
        d[target] -= 1
        
        self.dW += np.outer(d, x)
        delta = np.dot(self.W.T, d)
        
        return delta
    
    def loss_func(self):
        return sum(-np.log(y[target]) for target, y in zip(self.targets, reversed(self.pred)))

# 2. Seq2seq

In [6]:
from numpy.random import randint

EOS = 0
HIDDEN_SIZE = 10; EMBED_SIZE = 10; 
INPUT_SIZE = 4; OUTPUT_SIZE = 4;
INIT_RANGE = 1.0; LEARNING_RATE = 0.7; CLIP_GRAD = 5.0

class Seq2seq:
    def __init__(self, input_size=INPUT_SIZE, outupt_size=OUTPUT_SIZE\
                 , hidden_size=HIDDEN_SIZE, embed_size=EMBED_SIZE\
                , lr=LEARNING_RATE, clip_grad=CLIP_GRAD, init_range=INIT_RANGE):
        
        input_layers = [
            Embedding(output_size, embed_size, init_range),
            LSTM(embed_size, hidden_size, init_range),
        ]
        
        output_layers = [
            Embedding(output_size, embed_size, init_range),
            LSTM(embed_size, hidden_size, init_range, previous=input_layers[1]),
            Softmax(hidden_size, output_size, init_range)
        ]
        
        self.input_layers, self.output_layers = input_layers, output_layers
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.input_size = input_size
        self.output_size = output_size
        self.lr = lr
        self.clip_grad = clip_grad
        
    def predict(self, X, max_length=10):
        # reset state
        for layer in self.input_layers:
            layer.initSequence()
        # process input sequence
        for x in X:
            h = x
            for layer in self.input_layers:
                h = layer.forward(h)
        
        # reset state
        for layer in self.output_layers:
            layer.initSequence()
            
        out = []
        token = EOS
        
        while len(out) < max_length:
            h = token
            for layer in self.output_layers:
                h = layer.forward(h)
                
            token = np.argmax(h)
            
            if token == EOS:
                break
                
            out.append(token)
            
    def train(self, X, Y):
        # reset state
        for layer in self.input_layers:
            layer.initSequence()
        
        # forward pass
        for x in X:
            h = x
            for layer in self.input_layers:
                h = layer.forward(h)
        
        # reset state
        for layer in self.output_layers:
            layer.initSequence()
        
        for y in [EOS] + Y:
            h = y
            for layer in self.output_layers:
                h = layer.forward(h)
                
        # backward pass
        for y in reversed(Y + [EOS]):
            delta = y 
            for layer in reversed(self.output_layers):
                delta = layer.backward(delta)
                
        for x in reversed(X):
            delta = np.zeros(self.hidden_size)
            for layer in reversed(self.input_layers):
                delta = layer.backward(delta)
            
        grad_norm = 0
        
        for layer in self.input_layers + self.output_layers:
            for name, param, grad in layer.params:
                grad_norm += (grad ** 2).sum()
        
        grad_norm = np.sqrt(grad_norm)
        
        for layer in self.input_layers + self.output_layers:
            for name, param, grad in layer.params:
                if grad_norm > self.clip_grad:
                    grad /= grad_norm/self.clip_grad
                param -= self.lr * grad
                
        return self.output_layers[-1].loss_func()

# seq2seq -> attention -> self-attention
- reference
    -  https://medium.com/@bgg/seq2seq-pay-attention-to-self-attention-part-1-d332e85e9aad
    - https://medium.com/@bgg/seq2seq-pay-attention-to-self-attention-part-2-cf81bf32c73d
- <"Neural Machine Translation by Jointly Learning to Align and Translate">
- seq2seq 특징은,
    - fixed length의 context vector: encoder의 마지막 hidden state의 결과물
    - decoder의 h_t와 가장 관련있는 encoder의 h_n 파악
- attention 특징은,
    - N input word에 대하여 N개의 context vector가 존재
    - context vector는 input sequence의 hidden state의 합계
    - attention score은 sum of hidden state(attention score에 의해 가중곱)
- attention 모델의 단점은, 
    - 병렬이 안됨
    - source & target sentence의 attention반영 안됨. 즉 source & target에서 계산된 hidden로 attention 구함
- transformer의 특징,
    - self-attention, multi-head
state를 사용
- self attention으로 병렬 구현
    - RNN & CNN 사용안함

# make dataset

In [4]:
data = """
september 27, 1994           _1994-09-27
August 19, 2003              _2003-08-19
2/10/93                      _1993-02-10
10/31/90                     _1990-10-31
TUESDAY, SEPTEMBER 25, 1984  _1984-09-25
JUN 17, 2013                 _2013-06-17
april 3, 1996                _1996-04-03
October 24, 1974             _1974-10-24
AUGUST 11, 1986              _1986-08-11
February 16, 2015            _2015-02-16
October 12, 1988             _1988-10-12
6/3/73                       _1973-06-03
Sep 30, 1981                 _1981-09-30
June 19, 1977                _1977-06-19
OCTOBER 22, 2005             _2005-10-22
December 1, 2013             _2013-12-01
Wednesday, February 26, 1997 _1997-02-26
TUESDAY, SEPTEMBER 10, 1991  _1991-09-10
3/27/16                      _2016-03-27
APRIL 14, 1983               _1983-04-14
10/2/82                      _1982-10-02
APR 26, 1996                 _1996-04-26
12/21/79                     _1979-12-21
September 4, 2012            _2012-09-04
4/19/94                      _1994-04-19
5/14/95                      _1995-05-14
8/14/04                      _2004-08-14
mar 3, 2007                  _2007-03-03
sep 22, 2012                 _2012-09-22
11/21/10                     _2010-11-21
July 18, 2011                _2011-07-18
2/22/87                      _1987-02-22
april 15, 1987               _1987-04-15
july 12, 2012                _2012-07-12
june 12, 2001                _2001-06-12
Monday, September 8, 1997    _1997-09-08
Aug 29, 1994                 _1994-08-29
WEDNESDAY, MARCH 20, 2002    _2002-03-20
November 2, 1982             _1982-11-02
Apr 22, 2004                 _2004-04-22
April 7, 1992                _1992-04-07
9/25/16                      _2016-09-25
December 26, 2004            _2004-12-26
thursday, june 5, 2008       _2008-06-05
August 29, 2005              _2005-08-29
March 8, 1980                _1980-03-08
JULY 10, 2013                _2013-07-10
9/30/04                      _2004-09-30
APR 24, 2005                 _2005-04-24
August 22, 1999              _1999-08-22
5/6/93                       _1993-05-06
OCT 22, 1975                 _1975-10-22
10/25/97                     _1997-10-25
6/27/76                      _1976-06-27
5/18/13                      _2013-05-18
November 3, 1996             _1996-11-03
3/10/81                      _1981-03-10
1/6/79                       _1979-01-06
january 7, 2013              _2013-01-07
apr 21, 2011                 _2011-04-21
may 9, 1973                  _1973-05-09
1/21/03                      _2003-01-21
jun 5, 1991                  _1991-06-05
12/27/83                     _1983-12-27
Saturday, January 29, 2005   _2005-01-29
JANUARY 17, 2005             _2005-01-17
jul 22, 2014                 _2014-07-22
6/29/11                      _2011-06-29
Oct 13, 1996                 _1996-10-13
Thursday, August 24, 1989    _1989-08-24
1/28/71                      _1971-01-28
Saturday, January 8, 2005    _2005-01-08
Thursday, May 20, 1982       _1982-05-20
January 3, 1991              _1991-01-03
1/6/16                       _2016-01-06
"""

In [25]:
qes, ans = [], []
for line in data.split('\n')[1:-1]:
    idx = line.find('_')
    qes.append(line[:idx])
    ans.append(line[idx:])

id_to_char = {}
char_to_id = {}
for idx, _ in enumerate(qes):
    q, a = qes[idx], ans[idx]
    
    chars = list(q)
    for i, char in enumerate(chars):
        if char not in char_to_id:
            temp_id = len(char_to_id)
            char_to_id[char] = temp_id
            id_to_char[temp_id] = char
    
    chars = list(a)
    for i, char in enumerate(chars):
        if char not in char_to_id:
            temp_id = len(char_to_id)
            char_to_id[char] = temp_id
            id_to_char[temp_id] = char

In [19]:
x_shape = (len(qes), len(qes[0]))
t_shape = (len(ans), len(ans[0]))
x = np.zeros(x_shape, dtype=np.int)
t = np.zeros(t_shape, dtype=np.int)

In [27]:
for i, sentence in enumerate(qes):
    x[i] = [char_to_id[c] for c in list(sentence)] # [0,1,2,3,...]
for i, sentence in enumerate(ans):
    t[i] = [char_to_id[c] for c in list(sentence)]

In [29]:
indice = np.arange(len(x))
np.random.shuffle(indice)
x = x[indice]
t = t[indice]

In [32]:
split_at = len(x) - len(x) // 10
(x_train, x_test) = x[:split_at], x[split_at:]
(t_train, t_test) = t[:split_at], t[split_at:]

In [34]:
x_train

array([[52, 18, 46, ...,  7,  7,  7],
       [11,  8, 22, ...,  7,  7,  7],
       [11, 22, 36, ...,  7,  7,  7],
       ...,
       [27,  1, 41, ...,  7,  7,  7],
       [ 0,  1,  2, ...,  7,  7,  7],
       [17, 29, 32, ...,  7,  7,  7]])

In [36]:
# reference: https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import torch
import torch.nn as nn
import torch.optim as optim
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_size, n_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_size, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, data):
        embedded = self.dropout(self.embedding(data))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cel

In [37]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_size, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_size, n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(hidden_size, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, inputs, hidden, cell):
        inputs = inputs.unsqueeze(0) # [1, batch_size]
        embedded = self.dropout(self.embedding(inputs))
        outputs, (hidden, cell) = self.rnn(embedded)
        
        prediction = self.fc_out(outputs.squeeze(0))
        return prediction, hidden, cell